In [1]:
#
# Arquivo: agente_iot.py
# Descrição: Agente especializado em monitorar dados de dispositivos IoT em tempo real e detectar anomalias.
#
import time
import numpy as np
from datetime import datetime
from collections import deque

print("Dependências importadas com sucesso.")

def simular_dispositivo_iot(paciente_em_risco=False):
    """
    Um gerador que simula um dispositivo IoT (smartwatch) enviando dados a cada 2 segundos.
    Ele produz dados normais na maior parte do tempo, mas pode gerar anomalias.
    
    :param paciente_em_risco: Se True, aumenta a probabilidade de gerar anomalias.
    :yield: Um dicionário com os dados da leitura.
    """
    freq_cardiaca_base = 75
    spo2_base = 98
    
    while True:
        # Gera dados base com um pouco de ruído normal
        fc = np.random.normal(freq_cardiaca_base, 5)
        spo2 = np.random.normal(spo2_base, 0.5)
        
        # Chance de gerar uma anomalia
        chance_anomalia = 0.15 if paciente_em_risco else 0.05
        
        if np.random.rand() < chance_anomalia:
            tipo_anomalia = np.random.choice(['taquicardia', 'bradicardia', 'hipoxia', 'queda_subita_fc'])
            print(f"    (Simulador: Gerando anomalia do tipo '{tipo_anomalia}'...)")
            
            if tipo_anomalia == 'taquicardia':
                fc = np.random.normal(140, 10) # Frequência muito alta
            elif tipo_anomalia == 'bradicardia':
                fc = np.random.normal(40, 5)  # Frequência muito baixa
            elif tipo_anomalia == 'hipoxia':
                spo2 = np.random.normal(88, 2) # Saturação de oxigênio baixa
            elif tipo_anomalia == 'queda_subita_fc':
                # Simula uma queda brusca, o agente deve detectar a variação
                freq_cardiaca_base = 50 

        # Garante que os valores fiquem dentro de um intervalo razoável
        fc = np.clip(fc, 30, 200)
        spo2 = np.clip(spo2, 80, 100)
        
        leitura = {
            "timestamp": datetime.now(),
            "heart_rate_bpm": int(fc),
            "spo2_percent": round(spo2, 1)
        }
        
        yield leitura
        time.sleep(2) # Simula o intervalo entre leituras


class AgenteIoT:
    """
    Agente que monitora um fluxo de dados de saúde e detecta anomalias
    com base em regras pré-definidas.
    """
    def __init__(self, paciente_id: str, limiares: dict):
        """
        Inicializa o agente com os limiares específicos do paciente.
        
        :param paciente_id: Identificador do paciente.
        :param limiares: Dicionário com os valores críticos. Ex:
                         {'hr_max': 100, 'hr_min': 50, 'spo2_min': 92, 'hr_delta_max': 30}
        """
        self.paciente_id = paciente_id
        self.limiares = limiares
        # Usa um deque para armazenar o histórico recente de leituras (ex: últimos 10)
        self.historico_recente = deque(maxlen=10)
        print(f"Agente IoT inicializado para o paciente '{self.paciente_id}' com limiares definidos.")

    def _verificar_anomalia(self, dados_atuais: dict) -> (str, str):
        """
        Lógica interna para verificar se os dados atuais representam uma anomalia.
        Retorna uma tupla (tipo_alerta, mensagem) se uma anomalia for encontrada, senão None.
        """
        fc = dados_atuais['heart_rate_bpm']
        spo2 = dados_atuais['spo2_percent']

        # 1. Verificação de limiares absolutos
        if fc > self.limiares['hr_max']:
            return "Taquicardia", f"Frequência cardíaca ({fc} bpm) acima do limiar máximo ({self.limiares['hr_max']} bpm)."
        if fc < self.limiares['hr_min']:
            return "Bradicardia", f"Frequência cardíaca ({fc} bpm) abaixo do limiar mínimo ({self.limiares['hr_min']} bpm)."
        if spo2 < self.limiares['spo2_min']:
            return "Hipoxia", f"Saturação de oxigênio ({spo2}%) abaixo do limiar crítico ({self.limiares['spo2_min']}%)."

        # 2. Verificação de mudanças súbitas (se houver histórico suficiente)
        if len(self.historico_recente) > 1:
            fc_anterior = self.historico_recente[-1]['heart_rate_bpm']
            delta_fc = abs(fc - fc_anterior)
            if delta_fc > self.limiares['hr_delta_max']:
                return "Variação Súbita de FC", f"Variação súbita da frequência cardíaca detectada (de {fc_anterior} para {fc} bpm)."

        return None, None # Nenhuma anomalia

    def monitorar_ponto_de_dados(self, ponto_de_dados: dict) -> dict:
        """
        Recebe um único ponto de dados, o analisa e o adiciona ao histórico.
        Este é o principal ponto de entrada para integração.
        """
        tipo_alerta, mensagem = self._verificar_anomalia(ponto_de_dados)
        
        # Adiciona a leitura atual ao histórico para futuras análises de variação
        self.historico_recente.append(ponto_de_dados)
        
        if tipo_alerta:
            return {
                "status": "alerta",
                "paciente_id": self.paciente_id,
                "tipo_alerta": tipo_alerta,
                "motivo": mensagem,
                "dados_atuais": ponto_de_dados
            }
        else:
            return {
                "status": "normal",
                "paciente_id": self.paciente_id,
                "dados_atuais": ponto_de_dados
            }

# --- FUNÇÃO DE EXECUÇÃO E DEMONSTRAÇÃO ---

if __name__ == "__main__":
    print("="*60)
    print("DEMONSTRAÇÃO DO AGENTE IOT EM TEMPO REAL")
    print("="*60)
    
    # 1. Configurar o paciente e o agente
    id_paciente_demo = "Paciente-007"
    limiares_criticos = {
        'hr_max': 100,      # Batimentos por minuto (acima de)
        'hr_min': 50,       # Batimentos por minuto (abaixo de)
        'spo2_min': 92,     # Saturação de oxigênio (abaixo de)
        'hr_delta_max': 30  # Mudança máxima de FC entre duas leituras
    }
    
    agente_iot = AgenteIoT(id_paciente_demo, limiares_criticos)
    
    # 2. Iniciar o simulador de dispositivo
    # Vamos simular um paciente com maior risco para forçar anomalias
    fluxo_de_dados_iot = simular_dispositivo_iot(paciente_em_risco=True)
    
    print("\nIniciando monitoramento... (Pressione Ctrl+C para parar)\n")
    
    # 3. Loop de monitoramento em tempo real
    try:
        for i in range(50): # Monitora por 50 ciclos (aprox. 100 segundos)
            # Obter a leitura mais recente do dispositivo
            leitura_atual = next(fluxo_de_dados_iot)
            
            # O agente processa a leitura
            resultado = agente_iot.monitorar_ponto_de_dados(leitura_atual)
            
            # Exibir o status no console
            status = resultado['status'].upper()
            fc = resultado['dados_atuais']['heart_rate_bpm']
            spo2 = resultado['dados_atuais']['spo2_percent']
            timestamp = resultado['dados_atuais']['timestamp'].strftime('%H:%M:%S')
            
            print(f"[{timestamp}] Status: {status} | FC: {fc} bpm | SpO2: {spo2}%")
            
            # Se um alerta for gerado, tomar uma ação
            if resultado['status'] == 'alerta':
                print("-" * 60)
                print(f"🚨 ALERTA IMEDIATO GERADO PARA O PACIENTE: {resultado['paciente_id']} 🚨")
                print(f"   TIPO: {resultado['tipo_alerta']}")
                print(f"   MOTIVO: {resultado['motivo']}")
                print("-" * 60)
                
                # Em um sistema real, aqui você chamaria outro agente ou enviaria uma notificação.
                # Para a demonstração, vamos parar o monitoramento.
                break
                
    except KeyboardInterrupt:
        print("\nMonitoramento interrompido pelo usuário.")
    
    print("\nDemonstração concluída.")


Dependências importadas com sucesso.
DEMONSTRAÇÃO DO AGENTE IOT EM TEMPO REAL
Agente IoT inicializado para o paciente 'Paciente-007' com limiares definidos.

Iniciando monitoramento... (Pressione Ctrl+C para parar)

[12:10:46] Status: NORMAL | FC: 80 bpm | SpO2: 98.1%
[12:10:48] Status: NORMAL | FC: 71 bpm | SpO2: 97.0%
    (Simulador: Gerando anomalia do tipo 'hipoxia'...)
[12:10:50] Status: ALERTA | FC: 81 bpm | SpO2: 90.0%
------------------------------------------------------------
🚨 ALERTA IMEDIATO GERADO PARA O PACIENTE: Paciente-007 🚨
   TIPO: Hipoxia
   MOTIVO: Saturação de oxigênio (90.0%) abaixo do limiar crítico (92%).
------------------------------------------------------------

Demonstração concluída.


In [3]:
#
# Arquivo: agente_epidemiologico.py
# Descrição: Agente com RAG vetorial (FAISS) e RAG estruturado (Pandas/DuckDB).
#
import pandas as pd
import os
from typing import Dict, List, Any

# Ferramentas de busca, LLMs e RAG
from langchain_community.utilities import DuckDBLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings

print("Dependências do Agente Epidemiológico importadas com sucesso.")

class AgenteEpidemiologico:
    """
    Agente especializado em responder perguntas epidemiológicas usando três ferramentas:
    1. RAG Estruturado (Pandas): Para consultas em tabelas locais (ex: 'qual a média de idade?')
    2. RAG Vetorial (FAISS): Para perguntas conceituais em artigos (ex: 'quais as diretrizes?')
    3. Busca Externa (Tavily): Para informações da web em tempo real.
    """
    def __init__(self, dataset_path: str, vector_index_path: str, llm_model: str = "gpt-4o"):
        if not os.path.exists(dataset_path):
            raise FileNotFoundError(f"O arquivo de dataset (CSV) não foi encontrado em: {dataset_path}")
        if not os.path.exists(vector_index_path):
            raise FileNotFoundError(f"O índice vetorial FAISS não foi encontrado em: {vector_index_path}. Execute 'build_index.py' primeiro.")
            
        self.dataset_path = dataset_path
        self.vector_index_path = vector_index_path
        self.llm = ChatOpenAI(model=llm_model, temperature=0, api_key=os.getenv("OPENAI_API_KEY"))
        
        # Ferramenta 1: Buscador em Dados Locais (CSV)
        self.local_db_tool = self._criar_ferramenta_db_local()
        
        # Ferramenta 2: RAG Vetorial (FAISS)
        self.rag_tool = self._criar_ferramenta_rag_vetorial()
        
        # Ferramenta 3: Buscador Web Confiável (Tavily)
        self.web_search_tool = TavilySearchResults(max_results=3)
        
        # Cria o agente ReAct que decide qual ferramenta usar
        self.agent_executor = self._criar_agente_executor()
        print("Agente Epidemiológico (com RAG vetorial) inicializado com sucesso.")

    def _criar_ferramenta_db_local(self):
        """Cria a ferramenta para consultar dados estruturados (CSV/Pandas)."""
        print(f"Carregando dados de '{self.dataset_path}' para a ferramenta de dados locais...")
        
        def run_query(query: str) -> str:
            """Executa uma consulta no formato Pandas `query` no CSV."""
            try:
                df = pd.read_csv(self.dataset_path)
                result = df.query(query)
                return f"Resultado da consulta local:\n{result.to_markdown()}"
            except Exception as e:
                return f"Erro ao executar a consulta local: {e}. Tente uma sintaxe de query do Pandas."

        return Tool(
            name="BuscadorDeDadosLocais",
            func=run_query,
            description="""
            Use esta ferramenta para perguntas estatísticas sobre dados locais estruturados (CSV).
            A entrada deve ser uma query no formato do método `query` do Pandas (ex: 'age > 50 and diabetes == 1').
            Use apenas para perguntas sobre contagens, médias, prevalências, etc., que estão no CSV.
            """
        )

    def _criar_ferramenta_rag_vetorial(self):
        """Cria a ferramenta de RAG que busca em documentos (índice FAISS)."""
        print(f"Carregando índice vetorial FAISS de '{self.vector_index_path}'...")
        try:
            # Carrega o modelo de embedding
            model_name = "pritamdeka/S-PubMedBert-MS-MARCO"
            embeddings = SentenceTransformerEmbeddings(model_name=model_name)
            
            # Carrega o índice FAISS salvo
            db = FAISS.load_local(self.vector_index_path, embeddings, allow_dangerous_deserialization=True)
            retriever = db.as_retriever(search_kwargs={"k": 3}) # Retorna os 3 chunks mais relevantes
            
            def run_rag_query(query: str) -> str:
                """Executa uma busca semântica no índice vetorial."""
                docs = retriever.invoke(query)
                return f"Contexto encontrado nos documentos locais:\n" + "\n---\n".join([doc.page_content for doc in docs])

            return Tool(
                name="BuscadorDeArtigosMedicos",
                func=run_rag_query,
                description="""
                Use esta ferramenta para perguntas conceituais ou sobre diretrizes médicas,
                como 'quais são os sintomas de...', 'qual o tratamento para...', 
                'o que diz a diretriz sobre...'.
                Busca em artigos médicos e diretrizes salvas localmente.
                """
            )
        except Exception as e:
            print(f"ERRO CRÍTICO ao carregar o índice FAISS: {e}")
            print("Execute 'build_index.py' para criar o índice.")
            # Retorna uma ferramenta "vazia" para não quebrar o agente
            return Tool(
                name="BuscadorDeArtigosMedicos",
                func=lambda q: "Erro: Índice FAISS não carregado.",
                description="Erro: Índice FAISS não carregado."
            )

    def _criar_agente_executor(self):
        """
        Cria o agente principal que orquestra o uso das ferramentas.
        """
        tools = [self.local_db_tool, self.rag_tool, self.web_search_tool]
        
        prompt_template = """
        Você é um assistente de pesquisa epidemiológica de alto nível. Responda à seguinte pergunta da forma mais precisa possível.
        Você tem três ferramentas à sua disposição:

        1. `BuscadorDeDadosLocais`: Use para perguntas estatísticas sobre dados locais (ex: 'qual a média de idade?', 'quantos pacientes fumam?'). A entrada deve ser uma query Pandas.
        2. `BuscadorDeArtigosMedicos`: Use para perguntas conceituais ou sobre diretrizes (ex: 'quais os sintomas de...', 'qual o tratamento recomendado para...').
        3. `tavily_search_results`: Use como último recurso se as ferramentas locais não fornecerem uma resposta, ou para informações muito recentes (ex: 'notícias de hoje').

        **Prioridade:** Tente sempre usar `BuscadorDeDadosLocais` ou `BuscadorDeArtigosMedicos` primeiro.

        Use o seguinte formato:

        Pergunta: a pergunta de entrada que você precisa responder
        Pensamento: Devo analisar a pergunta. É uma consulta estatística (BuscadorDeDadosLocais), uma consulta conceitual (BuscadorDeArtigosMedicos) ou uma busca geral (tavily)?
        Ação: a ação a ser tomada, deve ser uma de [{tool_names}]
        Entrada da Ação: a entrada para a ação
        Observação: o resultado da ação
        ... (este Pensamento/Ação/Entrada/Observação pode se repetir N vezes)
        Pensamento: Agora eu sei a resposta final
        Resposta Final: a resposta final para a pergunta original. Sempre cite sua fonte (seja 'Dados Locais', 'Artigos Médicos Locais' ou uma URL da web).

        Comece!

        Pergunta: {input}
        Pensamento:{agent_scratchpad}
        """
        
        prompt = PromptTemplate.from_template(prompt_template)
        
        agent = create_react_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

    def analisar(self, pergunta: str) -> Dict[str, Any]:
        """
        Ponto de entrada principal para analisar uma pergunta epidemiológica.
        """
        print(f"\n>>> [Agente Epidemiológico] Recebendo pergunta: '{pergunta}'")
        try:
            resultado = self.agent_executor.invoke({"input": pergunta})
            return {
                "pergunta": pergunta,
                "resposta": resultado.get("output", "Não foi possível obter uma resposta."),
                "fonte": "Híbrida (IA + Ferramentas)"
            }
        except Exception as e:
            print(f"Ocorreu um erro durante a análise epidemiológica: {e}")
            return {
                "pergunta": pergunta,
                "resposta": "Ocorreu um erro ao processar sua pergunta.",
                "error": str(e)
            }


if __name__ == '__main__':
    # --- Demonstração de Uso ---
    
    # Verifique se as chaves de API estão configuradas
    if not os.getenv("OPENAI_API_KEY") or not os.getenv("TAVILY_API_KEY"):
        print("ERRO: Configure suas variáveis de ambiente OPENAI_API_KEY e TAVILY_API_KEY.")
        exit()

    # 1. Preparar os dados (Execute build_index.py primeiro!)
    dataset_filename = "synthetic_diabetes_data.csv"
    index_folder = "faiss_index"

    if not os.path.exists(index_folder) or not os.path.exists(dataset_filename):
        print("ERRO: Arquivos de dados ou índice não encontrados.")
        print("Por favor, execute 'build_index.py' e o script anterior para criar 'synthetic_diabetes_data.csv' primeiro.")
        exit()
    
    # 2. Inicializar o agente
    agente = AgenteEpidemiologico(dataset_path=dataset_filename, vector_index_path=index_folder)
    
    # 3. Fazer perguntas de teste
    
    # Pergunta 1: Teste do RAG Estruturado (Pandas)
    print("\n" + "="*50)
    pergunta_local = "Quantos pacientes com idade acima de 60 anos (age > 60) têm diabetes (diabetes == 1)?"
    resultado1 = agente.analisar(pergunta_local)
    print("\n--- RESULTADO 1 (Busca Estruturada) ---")
    print(f"Resposta: {resultado1['resposta']}")
    print("="*50)

    # Pergunta 2: Teste do RAG Vetorial (FAISS)
    print("\n" + "="*50)
    pergunta_conceitual = "O que este documento diz sobre o tratamento de diabetes?"
    resultado2 = agente.analisar(pergunta_conceitual)
    print("\n--- RESULTADO 2 (RAG Vetorial) ---")
    print(f"Resposta: {resultado2['resposta']}")
    print("="*50)

    # Pergunta 3: Teste da Busca Externa (Tavily)
    print("\n" + "="*50)
    pergunta_externa = "Quais são as últimas notícias sobre a cura do diabetes tipo 1? site:cdc.gov"
    resultado3 = agente.analisar(pergunta_externa)
    print("\n--- RESULTADO 3 (Busca Externa) ---")
    print(f"Resposta: {resultado3['resposta']}")
    print("="*50)


ModuleNotFoundError: No module named 'langchain_community'